In [62]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [63]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [64]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  # One Hot encoding
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)



Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [65]:
# print max(train_dataset[0]), min(train_dataset[0])
train_dataset += 0.5
valid_dataset += 0.5
test_dataset += 0.5

In [77]:
# auto encoder 
batch_size = 128
num_hidden = 1024
beta = 0.2
rho = 0.05

def x_entropy(output, target):
  with tf.name_scope("xentropy_loss"):
      cross_entropy = tf.add(tf.mul(tf.log(output, name='log_output'),target),
                             tf.mul(tf.log(1 - output),(1 - target)))
      return -1 * tf.reduce_mean(tf.reduce_sum(cross_entropy, 1), name='xentropy_mean')

graph = tf.Graph()
with graph.as_default():
  #Inputs
  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  Xinit = 4 * np.sqrt(6.0 / (image_size * image_size + batch_size))
  hidden_weights = tf.Variable(tf.random_uniform([image_size * image_size, num_hidden], -1*Xinit, Xinit))
  hidden_biases = tf.Variable(tf.zeros([num_hidden]))
  out_biases = tf.Variable(tf.zeros([image_size * image_size]))

  classifier_weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
  classifier_biases = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  def autoencode(data):
    a1 = tf.nn.sigmoid(tf.matmul(data, hidden_weights) + hidden_biases)
    rho_hat = tf.reduce_mean(a1,0)
    kl = tf.add(tf.mul(rho, tf.log(rho / rho_hat)),
                tf.mul(1-rho, tf.log((1-rho) / (1-rho_hat))))
    return tf.nn.sigmoid(tf.matmul(a1, hidden_weights, transpose_b = True) + out_biases), tf.reduce_sum(kl)

  encoded, kl = autoencode(tf_train_dataset)
#   autoencoder_loss = x_entropy(a2, tf_train_dataset)
  autoencoder_loss = tf.sqrt(tf.reduce_mean(tf.square(encoded-tf_train_dataset))) + kl
    
  logits = tf.matmul(encoded, classifier_weights) + classifier_biases
  classify_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits,tf_train_labels))
  
  # Optimizer.
  ae_optimizer = tf.train.GradientDescentOptimizer(0.3).minimize(autoencoder_loss)

  global_step = tf.Variable(0, trainable=False)
  learning_rate = tf.train.exponential_decay(0.5, global_step, 500, 0.8, staircase=True)
  cl_optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(classify_loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction, _ = autoencode(tf_valid_dataset)
  test_prediction, _ = autoencode(tf_test_dataset)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_prediction, classifier_weights) + classifier_biases)
  test_prediction = tf.nn.softmax(tf.matmul(test_prediction, classifier_weights) + classifier_biases)

In [78]:
num_steps = 5001

# def encoding_accuracy(pred, labels):
#     return 100.0 * sum([np.array_equal(pred[i],labels[i]) for i in xrange(len(labels))]) / len(labels)

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  # ae training 
  for step in range(num_steps):
    offset = (step * batch_size) % (train_dataset.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    feed_dict = {tf_train_dataset : batch_data}
    _, l, predictions = session.run(
      [ae_optimizer, autoencoder_loss, encoded], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))

  # classifier training
  for step in range(num_steps):
    offset = (step * batch_size) % (train_dataset.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels: batch_labels}
    _, l, predictions = session.run(
      [cl_optimizer, classify_loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 669.628235
Minibatch loss at step 500: 2.222730
Minibatch loss at step 1000: 2.275139
Minibatch loss at step 1500: 3.266211
Minibatch loss at step 2000: 2.438263
Minibatch loss at step 2500: 1.656603
Minibatch loss at step 3000: 1.369729
Minibatch loss at step 3500: 1.427632
Minibatch loss at step 4000: 1.011026
Minibatch loss at step 4500: 0.919349
Minibatch loss at step 5000: 1.383479
Minibatch loss at step 0: 10.960165
Minibatch accuracy: 10.2%
Validation accuracy: 15.0%
Minibatch loss at step 500: 0.678513
Minibatch accuracy: 80.5%
Validation accuracy: 82.5%
Minibatch loss at step 1000: 0.719513
Minibatch accuracy: 82.8%
Validation accuracy: 83.2%
Minibatch loss at step 1500: 0.365552
Minibatch accuracy: 89.1%
Validation accuracy: 85.3%
Minibatch loss at step 2000: 0.348819
Minibatch accuracy: 91.4%
Validation accuracy: 85.5%
Minibatch loss at step 2500: 0.369534
Minibatch accuracy: 87.5%
Validation accuracy: 86.1%
Minibatch loss at step 3000: 